In [104]:
import pandas as pd
import requests
import time

In [105]:
# Riot Developer Portal에서 받은 API KEY입니다.
# 해당 값을 포함한 모든 종류의 KEY는 코드에 직접적으로 노출되지 않도록 하는 것이 좋습니다.
api_key = 'RGAPI-80e3307a-8620-40dc-bcef-b78a866b75aa'

# 본인 소환사명입니다.
summoner_name = 'midplayerorafk'

In [106]:
# SUMMONER-V4 API URL
requesturl = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+summoner_name+\
'?api_key='+api_key

In [107]:
# puuid입니다.
summoner_puuid = requests.get(requesturl).json()['puuid']

In [108]:
games_list

['KR_5507541819',
 'KR_5507469774',
 'KR_5506990572',
 'KR_5506959118',
 'KR_5506978025',
 'KR_5506889686',
 'KR_5506918758',
 'KR_5505993936',
 'KR_5505947266',
 'KR_5504648486',
 'KR_5503204743',
 'KR_5502937995',
 'KR_5469854606',
 'KR_5469812211',
 'KR_5469699850',
 'KR_5464062998',
 'KR_5463869236',
 'KR_5463836202',
 'KR_5457760082',
 'KR_5452808998',
 'KR_5452747487',
 'KR_5452793372',
 'KR_5452698497',
 'KR_5439654128',
 'KR_5439611313',
 'KR_5439467495',
 'KR_5434604410',
 'KR_5434621898',
 'KR_5434588684',
 'KR_5434585636',
 'KR_5425061942',
 'KR_5423291286',
 'KR_5423109935',
 'KR_5418791148',
 'KR_5418668424',
 'KR_5418695763',
 'KR_5418634762',
 'KR_5417310543',
 'KR_5416999188',
 'KR_5410978939',
 'KR_5410918929',
 'KR_5410916087',
 'KR_5410963547',
 'KR_5404612925',
 'KR_5404524650',
 'KR_5404551967',
 'KR_5404475794',
 'KR_5400506815',
 'KR_5400561855',
 'KR_5400448738',
 'KR_5400455630',
 'KR_5400384457',
 'KR_5400359462',
 'KR_5400337218',
 'KR_5398853152',
 'KR_53988

In [109]:
# 솔랭 게임 목록 가져오기

games_list = []
index = 0
while True:
    requesturl = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/'+summoner_puuid+\
'/ids?queue=450&start='+str(index)+'&count=100&api_key='+api_key
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_list_new = response.json()
            break
    
    games_list.extend(game_list_new)
    
    if len(game_list_new) != 100:
        break
        
    index += 100
    time.sleep(1)

In [110]:
len(games_list)

229

In [111]:
# 솔랭 게임 각각에 대해서 API 호출 및 row binding
games_df = pd.DataFrame()
for matchId in games_list:
    requesturl = 'https://asia.api.riotgames.com/lol/match/v5/matches/'+matchId+\
    '?api_key='+api_key
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_json = response.json()
            break
    game_df = pd.DataFrame(game_json['info']['participants'])[['puuid','championId','win','kills','deaths','assists']]
    game_df = game_df[game_df.puuid == summoner_puuid][['championId','win','kills','deaths','assists']]
        
    games_df = pd.concat([games_df,game_df])
    time.sleep(1)

In [112]:
pd.DataFrame(game_json['info']['participants'])[['puuid','championId','win','kills','deaths','assists']]

,puuid,championId,win,kills,deaths,assists
0,ZRQMRm9bJeet28u08d7A0CiShrV4hJMljeeewRsxGE-5mO...,68,True,6,11,18
1,reuOGnkyglJJHpnuUvjd3ueI4JlaG-fPDHNC4UqKMhIVt4...,37,True,8,8,24
2,jL9HMbPJLbrBwQ8pMJoHx4Rcc9yGkidsUyH7GdCyybF4nc...,51,True,14,7,15
3,ev2GocL9UsfRa-UUCtHdK-UGRPHz8YiPuo_qinOCPNdZ50...,106,True,4,10,27
4,DZRY_oNQaK2USj1lo7NimQbSnf-SsWipgM4stwRi3J-RaL...,150,True,8,9,22
5,R7JXiyrjuSedkDNBUSJz4Oc0bLETSFCsnxC0-ZJ3oNTIy4...,517,False,12,9,19
6,Un_SaMU-YFPtA3PT5rPxWE-LG6Np5Gr2Iq5KRfHtIuofNs...,131,False,11,9,14
7,9YNBjGiiO8ZoErmjlzf5lAqW0TCreBhsdq2dyRUrGLCmJK...,4,False,12,8,21
8,kCi6LQm1YhLr1979yRwW5JVv4O_9nSO1GZgE9JSMrjmOY8...,22,False,7,5,26
9,EM9sp1RvGHffKFY8G3M0xzDXRid_BjpTJo6QkF2isKe-2Y...,12,False,3,9,34


In [113]:
games_df = games_df.groupby(['championId'],as_index=False).\
apply(lambda x: pd.Series({'win_count':sum(x.win),
                           'count':len(x),
                           'kills':sum(x.kills),
                           'deaths':sum(x.deaths),
                           'assists':sum(x.assists)
                          }))

In [114]:
# 챔피언 Key값에 대한 Value 가져오기
last_version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
champion_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/"+last_version+"/data/ko_KR/champion.json")

In [115]:
# Json 파일을 DataFrame으로 변환
champion_df = pd.DataFrame(champion_constant.json()['data']).T[['key','name']]

# 변수형 문자 -> 숫자
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [116]:
games_df = games_df.merge(champion_df,
                          how = 'left',
                          left_on = 'championId',
                          right_on = 'key'
                         )[['name','count','win_count','kills','deaths','assists']]

In [117]:
games_df['WR'] = games_df['win_count'] / games_df['count']
games_df['KDA'] = (games_df['kills']+games_df['assists']) / games_df['deaths']

In [118]:
pd.set_option('display.max_row', None)

In [120]:
games_df.sort_values('count',ascending = False)[['name','WR','count','KDA']]

,name,WR,count,KDA
29,카르마,0.777778,9,5.428571
12,트리스타나,0.142857,7,2.025641
25,문도 박사,0.333333,6,2.500000
91,세트,0.833333,6,2.676056
77,세나,0.600000,5,4.552632
56,바루스,0.600000,5,5.027027
31,트런들,0.200000,5,2.420000
69,세라핀,0.800000,5,5.100000
75,진,0.600000,5,4.216216
45,이즈리얼,0.800000,5,3.578947
